In [ ]:
#param
file = open('/Users/lpr/Data/FAST/example_1092/1092.param')
temp = file.readlines()
for stdfile in stdima_order:
    goodsn_path = '/Users/lpr/Data/FAST/goodsn'
    image_index = stdfile[stdfile.index('_',7)+1:stdfile.index('.fits')]
    file = open(goodsn_path+'/'+image_index+'/'+image_index+'.param','w')
    for num in range(0,len(temp)):
        if num < 2:
            name,value = temp[num].split('=')
            value = ' \''+image_index+'\''
            temp[num] = name+'='+value+'\n'
        file.write(temp[num])
#filter
file = open('/Users/lpr/Data/FAST/example_1092/FILTER.RES.v7.R300')
temp = file.readlines()
goodsn_path = '/Users/lpr/Data/FAST/goodsn'
for stdfile in stdima_order:
    image_index = stdfile[stdfile.index('_',7)+1:stdfile.index('.fits')]
    file = open(goodsn_path+'/'+image_index+'/FILTER.RES.v7.R300','w')
    for num in range(0,len(temp)):
        file.write(temp[num])
#flux catalog
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
import os
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.optimize import curve_fit

image_folder_path = '/Users/lpr/Data/fits/expdata/HST/goodsn_all/multiband_cutimage'
image_path = image_folder_path+'/goodsn_f160w'
fast_path = '/Users/lpr/Data/FAST'
filelist = os.listdir(image_folder_path)
dirlist = []
for directory in filelist:
    if directory.startswith('goodsn'):
        dirlist.append(directory)
        
# we use stdima to get image index, such as 597, 1092......
filelist = os.listdir(image_folder_path+'/goodsn_f098m')
stdima_order = []
for file in filelist:
    if file.endswith('.fits'):
        stdima_order.append(file)

radec_table = fits.open('/Users/lpr/Data/fits/expdata/HST/goodsn_all/radec_table.fits')[1]
def subt_backnoise(image):
    hdu = np.transpose(image.data)
    hdu1 = hdu[np.where((hdu>-0.01)&(hdu<0.01))]
    noise = np.median(hdu1)
    return(hdu-noise,noise)
# draw SED diagram
flux_list = {}
for stdfile in stdima_order:
    if '5297' in stdfile:
        image_index = stdfile[stdfile.index('_',7)+1:stdfile.index('.fits')]
        galaxy_name = 'goodsn_f160w_'+image_index+'.fits'
        cata_index = np.where(radec_table.data['galaxy_id']==int(image_index))[0]
        image_160 = fits.open(image_path+'/'+galaxy_name)[0]
    #        exptime_160 = float(image_160.header['EXPTIME'])
    #        elec_to_flux_160 = image_160.header['PHOTFNU']*10**6
        elec_to_flux_160 = image_160.header['PHOTFLAM']*image_160.header['PHOTPLAM']**2*1e6/3e-5
        image_160.header['CTYPE1'] = 'RA---TAN-SIP'
        image_160.header['CTYPE2'] = 'DEC--TAN-SIP'
        his = image_160.header['HISTORY'][1]
        cut_range = np.array([int(his[his.index('[')+2:his.index(':',16)]),int(his[his.index(',')+3:his.index(':',his.index(','))])])
        ra,dec = float(radec_table.data['galaxy_160_ra'][cata_index]),float(radec_table.data['galaxy_160_dec'][cata_index])
        redshift = str(float(radec_table.data['catalog_redshift'][cata_index]))
        center = np.array(WCS(image_160.header).wcs_world2pix(ra,dec,0))-cut_range
        flux_160 = 0
        aftersubt_image_160,sky_160 = subt_backnoise(image_160)
        # from last step, image array has been transposed
        pixel_number = 0
        for num_x in range(0,aftersubt_image_160.shape[0]):
            for num_y in range(0,aftersubt_image_160.shape[1]):
                if np.sqrt((num_x-float(center[0]))**2+(num_y-float(center[1]))**2)<=4:
                    flux_160 += aftersubt_image_160[num_x][num_y]
                    pixel_number += 1
        err_160 = np.sqrt(pixel_number)*sky_160
    #        flux_list[1600] = flux_160*elec_to_flux_160,err_160*elec_to_flux_160
        flux_list[1600] = flux_160*elec_to_flux_160,err_160*elec_to_flux_160
        #============ append band in x axis of SED 
        x_axis = np.zeros(0) # band list
        x_axis = np.append(x_axis,1600)
        for band in dirlist:
            #============ next few steps is to do photometry in circle apertures
            if 'f160' not in band:
                image = fits.open(image_folder_path+'/'+band+'/'+band+'_'+image_index+'.fits')[0]
    #                elec_to_flux = image.header['PHOTFLAM']*image.header['PHOTPLAM']**2*1e6/(3e-5)
                elec_to_flux = image.header['PHOTFLAM']*image.header['PHOTPLAM']**2*1e6/3e-5
                aftersubt_image,sky = subt_backnoise(image)
                flux = 0
                for num_x in range(0,len(aftersubt_image)):
                    for num_y in range(0,len(aftersubt_image)):
                        if np.sqrt((num_x-float(center[0]))**2+(num_y-float(center[1]))**2)<=4:
                            flux += aftersubt_image[num_x][num_y]
                err = np.sqrt(pixel_number)*sky
                if '1' not in band[band.index('f')+1:band.index('f')+2] and '0'not in band[band.index('f')+1:band.index('f')+2]:
                    if flux>1e-10:
                        x_axis = np.append(x_axis,int(band[band.index('f')+1:-1]))
                        flux_list[int(band[band.index('f')+1:-1])] = flux*elec_to_flux,err*elec_to_flux
                else:
                    if flux>1e-10:
                        x_axis = np.append(x_axis,int(band[band.index('f')+1:-1])*10)
                        flux_list[int(band[band.index('f')+1:-1])*10] = flux*elec_to_flux,err*elec_to_flux
    #                if image.header['BUNIT'] == 'ELECTRONS/S':
    #                    aftersubt_image,sky = subt_backnoise(image)
    #                    flux = 0
    #                    for num_x in range(0,len(aftersubt_image)):
    #                        for num_y in range(0,len(aftersubt_image)):
    #                            if np.sqrt((num_x-float(center[0]))**2+(num_y-float(center[1]))**2)<=11:
    #                                flux += aftersubt_image[num_x][num_y]
    #                    err = np.sqrt(pixel_number)*sky
    #                    if '1' not in band[band.index('f')+1:band.index('f')+2] and '0'not in band[band.index('f')+1:band.index('f')+2]:
    #                        flux_list[int(band[band.index('f')+1:-1])] = flux*elec_to_flux,err*elec_to_flux
    #                    else:
    #                        flux_list[int(band[band.index('f')+1:-1])*10] = flux*elec_to_flux,err*elec_to_flux
    #                elif image.header['BUNIT'] == 'ELECTRONS': 
    #                    aftersubt_image,sky = subt_backnoise(image)
    # #                    aftersubt_image = aftersubt_image/image.header['EXPTIME']
    # #                    sky = sky/image.header['EXPTIME']
    #                    flux = 0
    #                    for num_x in range(0,len(aftersubt_image)):
    #                        for num_y in range(0,len(aftersubt_image)):
    #                            if np.sqrt((num_x-float(center[0]))**2+(num_y-float(center[1]))**2)<=11:
    #                                flux += aftersubt_image[num_x][num_y]
    #                    err = np.sqrt(pixel_number)*sky
    #                    if '1' not in band[band.index('f')+1:band.index('f')+2] and '0'not in band[band.index('f')+1:band.index('f')+2]:
    #                        flux_list[int(band[band.index('f')+1:-1])] = flux*elec_to_flux,err*elec_to_flux
    #                    else:
    #                        flux_list[int(band[band.index('f')+1:-1])*10] = flux*elec_to_flux,err*elec_to_flux
        #============ plot SED
        y_axis = np.zeros(0) # flux of every band
        errorbar = np.zeros(0)
        x_axis = np.sort(x_axis)
        for wavelength in x_axis:
            if wavelength in flux_list:
                y_axis = np.append(y_axis,flux_list[wavelength][0])
                errorbar = np.append(errorbar,flux_list[wavelength][1])
        # remove band without data
        x_axis = np.delete(x_axis,np.where(np.isnan(y_axis)))
        errorbar = np.delete(errorbar,np.where(np.isnan(y_axis)))
        y_axis = np.delete(y_axis,np.where(np.isnan(y_axis)))
    #        x_axis = np.delete(x_axis,np.where(y_axis<np.e**-3))
    #        errorbar = np.delete(errorbar,np.where(y_axis<np.e**-3))
    #        y_axis = np.delete(y_axis,np.where(y_axis<np.e**-3))
        fast_gala_path = fast_path+'/'+image_index+'_re'
        #fast_gala_path = fast_path+'/'+image_index
        os.system('mkdir '+fast_gala_path)
        print(y_axis)
        txt_row = '1 '
        txt_header = '# id '
        for num in range(0,y_axis.shape[0]):
            txt_row += str(y_axis[num])+' '+str(errorbar[num])+' '
            txt_header += str(int(x_axis[num]))+' '+str(int(x_axis[num]))+'err'+' '
    #        np.savetxt(sed_path+image_index+'.cat',y_axis.reshape(1,y_axis.shape[0]),header=str(x_axis),fmt='%e')
    #        np.savetxt(sed_path+image_index+'.cat',txt_row,header=txt_header,fmt='%e')
        txt_file = open(fast_gala_path+'/'+image_index+'_re.cat','w')
        #txt_file = open(fast_gala_path+'/'+image_index+'.cat','w')
        txt_file.write(txt_header+' z_spec'+'\n')
        txt_file.write(txt_row+' '+redshift)
        txt_file.close
        print(image_index+'done')
    #        plt.figure(figsize=(10,10))
    #        plt.errorbar(x_axis,y_axis,yerr=errorbar,fmt='.',mec='r',elinewidth=1,capsize=1,capthick=1)
    #        plt.xticks(x_axis,rotation=90)
    #        plt.yticks()
    #        plt.xlabel('$\lambda (nm)$')
    #        plt.ylabel('$f_{\lambda}(\mu Jy)$')
    #        plt.title(image_index+' SED')
    #        print(image_index+' SED is done')